## Creando inputs para el modelo AMPL a partir de archivo csv: Archivo .dat

En este cuadernillo se desarrolla el código para insertar los distintos parámetros al modelo. En vista de su practicidad, se decidió insertar los datos usando objetos de pandas creados con un archivo csv

In [ ]:
# Leemos los conjuntos